In [14]:
#packages
import sys
import re
import gensim
import numpy as np
import string
import pandas as pd
import os
import yahoo_finance
import yfinance as yf
import datetime
import matplotlib.pylab as plt
import json

In [365]:
# roots
baseRoot = r'D:\wiki_cleaner\Whole_wikipedia\results100\Articles\Articles'
finalRoot = r'D:\wiki_cleaner\Whole_wikipedia\results100'

In [366]:
# colections
allArticlesPerDate = {}
datePriceDictionary = {}
startDate = None

In [367]:
# convert string to date format
def convertToDateFromat(stringDate):
    if len(stringDate.split('.')) != 3:
        print('Wrong format of date: '+stringDate)
        raise
    return datetime.datetime.strptime(stringDate.split('.')[2]+'-'+stringDate.split('.')[1]+'-'+stringDate.split('.')[0], '%Y-%m-%d').date() 

In [368]:
def convertToDateFromatFromBitcoinApiFormat(stringDate):
    if len(stringDate.split('.')) != 3:
        print('Wrong format of date: '+stringDate)
        raise
    return str(i.date()).split('-')[2]+'.'+str(i.date()).split('-')[1]+'.'+str(i.date()).split('-')[0]

In [369]:
# create dictionary with key as datetime.date format and values as list of articles
def articlesSegregation(baseRoot):
    dateDictionary = {}
    articlesDictionary = {}
    for externalFolderName in os.listdir(baseRoot):
        for innerFileName in os.listdir(os.path.join(baseRoot, externalFolderName)):
            for fileName in os.listdir(os.path.join(os.path.join(baseRoot, externalFolderName), innerFileName)):            
                name = str(baseRoot+'/'+ externalFolderName+ '/' + innerFileName + '/' + fileName)
                if externalFolderName in dateDictionary:
                    dateDictionary[externalFolderName].append(name)
                elif externalFolderName not in dateDictionary:
                    dateDictionary[externalFolderName] = [name]  
    for date in dateDictionary:
        if date.split('.')[2][0] != '1' and date.split('.')[2][0] != '2':
            continue
        for article in dateDictionary[date]:
            if convertToDateFromat(date) in articlesDictionary:
                articlesDictionary[convertToDateFromat(date)].append(' '.join(open(article, 'r',errors='ignore',encoding="utf8").readlines()) + '\n')
            elif convertToDateFromat(date) not in articlesDictionary:
                articlesDictionary[convertToDateFromat(date)] = [' '.join(open(article, 'r',errors='ignore',encoding="utf8").readlines()) + '\n']
    return articlesDictionary

In [370]:
def createBitcoinSharesDataset(articlesDictionary,timePeriod):
    datePriceDictionary = {}
    # create connection object to bitcoin API
    bitcoinShares = yf.Ticker("BTC-USD")
    startDate = sorted(list(articlesDictionary.keys()))[0]
    # get historical market data for 5 year period
    historical_data = bitcoinShares.history(period=timePeriod)
    for i, row in historical_data.iterrows():
        if(i.date() >= startDate):
            datePriceDictionary[i.date()] = row['Close']
    return datePriceDictionary

In [371]:
# create dictionary with key as datetime.date format and values as list of articles
def prepareArticlesLabels(sharesDictionary):    
    labeledDictionary = {}
    sortedDict = sorted(list(sharesDictionary.keys()))
    for x in range(0, len(sortedDict)):
        if x == len(sortedDict) - 1:
            break
        if sharesDictionary[sortedDict[x]] > sharesDictionary[sortedDict[x+1]]:
            labeledDictionary[sortedDict[x+1]] = 'pos'
        else:
            labeledDictionary[sortedDict[x+1]] = 'neg'
    return labeledDictionary

In [372]:
def labelArticlesWitchMarketChange(segregatedArticles, markedDates):
    labeledArticleDictionary = {}
    for date in markedDates.keys():        
        if date in markedDates and date in segregatedArticles:
            labeledArticleDictionary[date] = [markedDates[date], segregatedArticles[date]]
        else:
            print('Missing dates: '+ str(date))
    return labeledArticleDictionary

In [373]:
def removeStopList(article):
    article = article.split() 
    stoplist = ['the', 'a', 'and', 'of', 'to', 'is', 'in', 'i', 'it', 'that', 'this', 'br', 'on', 'his', 'her', 'he', 'she', 'one',
           'all', 'an', 'at', 'by', 'all', 'an', 'or', 'its', 'they', 'just', 'out', 'just', 'some', 'if', 'see', 'my', 'me']
    for word in stoplist:
        if word in article:
            article = [value for value in article if value != word]
    return article

In [374]:
def removePunctuation(article):
    return " ".join(" ".join(article).translate(str.maketrans('','',string.punctuation)).lower().replace('\n',' ').split()) 

In [375]:
def createJson(article,label):
    start = '{"text":['
    end = '], "label": "{lab}"'.format(lab = label) + '}'
    middle = ''
    for word in article:
        middle += '"' + str(word) + '"' +','
    middle = middle[:-1]
    json = start+ middle + end + '\n'
    return json

In [376]:
def preprocessTextData(labeleDictionary,finalRoot):
    file_object = open(finalRoot + r'/result.json', 'a',errors='ignore',encoding="utf8")
    for key, value in labeleDictionary.items():
        article = labeleDictionary[key][1]
        label = labeleDictionary[key][0]
        article = removePunctuation(article)
        article = removeStopList(article)
        file_object.write(createJson(article,label))
        

In [12]:
def tryJson(root):
    file_object = open(root, 'r',errors='ignore',encoding="utf8")
    lines = file_object.readlines()
    for line in lines:        
        json.loads(line)

In [377]:
timePeriodForBitcoingAnalysing = "5y"
segregatedArticles = articlesSegregation(baseRoot)
bitcoinSharesPrice = createBitcoinSharesDataset(segregatedArticles,timePeriodForBitcoingAnalysing)
markedDates = prepareArticlesLabels(bitcoinSharesPrice)
labeledArticlesDictionary = labelArticlesWitchMarketChange(segregatedArticles, markedDates)
preprocessTextData(labeledArticlesDictionary,finalRoot)

Missing dates: 2020-04-21
Missing dates: 2020-04-22
Missing dates: 2020-04-23
Missing dates: 2020-04-25
